In [1]:
import torch
import os
import time
import json
## Load Config
with open('config/videos.json') as config_file:
    videos = json.load(config_file)
with open('config/name_to_url.json') as config_file:
    name_to_url = json.load(config_file)


In [2]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

torch.cuda.empty_cache()
device = "cuda:0" if torch.cuda.is_available() else "cpu"
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1", torch_dtype=torch.float16, attn_implementation="flash_attention_2").to(device)


# tokenizer = AutoTokenizer.from_pretrained("TabbyML/StarCoder-7B")
# model = AutoModelForCausalLM.from_pretrained("TabbyML/StarCoder-7B")

/home/ubuntu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.
Loading checkpoint shards: 100%|██████████| 2/2 [00:07<00:00,  3.68s/it]


In [3]:
# Variable Setup
shots = ["mixtral8x7b", "full-stack"]
torch.cuda.empty_cache()

# folder_no = len(os.listdir(f'data/outputs')) + 1
folder_no = 101
print(f"Folder Number: {folder_no}")
os.makedirs(f'data/outputs/{folder_no}/text', exist_ok=True)
os.makedirs(f'data/outputs/{folder_no}/embeddings', exist_ok=True)
os.makedirs(f'data/outputs/{folder_no}/results', exist_ok=True)

transcripts_dir = "data/transcripts/processed"
prompt_path = "data/prompts/prompt.txt"
oneshots_dir = "data/oneshots"
outputs_dir = f'data/outputs/{folder_no}'

prompt = open(prompt_path, "r").read()
# Write some metadata

with open(f'data/outputs/{folder_no}/metadata.txt', 'a') as f:
    if os.stat(f'data/outputs/{folder_no}/metadata.txt').st_size == 0:
        f.write(f"#####################\n###### METADATA ######\n#####################\n\n")
        f.write(f"Model: mistralai/Mistral-7B-Instruct-v0.1\n")
        f.write(f"Time: {time.time()}\n")
        f.write(f"Videos: {videos}\n")
        f.write(f"Shots: {shots}\n")
        f.write(f"Prompt: {prompt}\n")

Folder Number: 101


In [4]:
def abridge_transcript(transcript: str, chars: int) -> str:
    """ Abridge a transcript to a certain number of characters
    Args:
        transcript (str): The transcript to abridge
        chars (int): The number of characters to abridge to
    Returns:
        str: The abridged transcript
    """
    f_abridged_transcripts = open(f'data/outputs/{folder_no}/abridged_transcripts.txt', 'a')

    if len(transcript) <= chars:
        return transcript
    
    output_transcript = ""
    ## Use 20% of chars for the start
    output_transcript += transcript[:int(chars*0.2)]
    output_transcript += " "

    # evenly divide the characters in the middle of the transcript and extract in 6 10% chunks
    middle = transcript[int(chars*0.2):-int(chars*0.2)]
    transcript_chunk_size = len(middle) // 6
    abridged_chunk_size = int(chars * 0.1)
    for i in range(6):
        target_chunk_index = transcript_chunk_size * i + (transcript_chunk_size // 2)
        output_transcript += middle[target_chunk_index:target_chunk_index+abridged_chunk_size]
        output_transcript += " "

    ## Use 20% of chars for the end
    output_transcript += transcript[-int(chars*0.2):]
    f_abridged_transcripts.write(f"{output_transcript}\n\n")
    f_abridged_transcripts.close()
    return output_transcript



def build_message(shots: list, target: str, shots_char_limit: int, input_char_limit: int) -> list:
    """ Build a message for the model to generate from.
    Args:
        shots (list): List of shots to include in the message
        target (str): The target video to generate the message for
        start (int): The start of the transcript to include
        end (int): The end of the transcript to include
    Returns:
        list: A list of messages to send to the model
    """
    f_abridged_transcripts = open(f'data/outputs/{folder_no}/abridged_transcripts.txt', 'a')

    messages = []

    f_prompt = open(prompt_path, 'r', encoding='utf-8')
    f_target_transcript = open(f'{transcripts_dir}/{target}.txt', 'r', encoding='utf-8')
    
    prompt = f_prompt.read()
    for shot in shots:
        f_oneshot_transcript = open(f'{transcripts_dir}/{shot}.txt', 'r', encoding='utf-8')
        f_oneshot = open(f'{oneshots_dir}/{shot}.txt', 'r', encoding='utf-8')
        
        f_abridged_transcripts.write(f"#####################\nShot: {shot}\n#####################\n")
        messages.append({"role": "user", "content": f'Transcript:\n{abridge_transcript(f_oneshot_transcript.read(), shots_char_limit)}\nInstruction:\n{prompt}'})
        messages.append({"role": "assistant", "content": f_oneshot.read()})

        f_oneshot.close()
        f_oneshot_transcript.close()

    ## Append final message
    f_abridged_transcripts.write(f"#####################\nTarget: {target}\n#####################\n")
    messages.append({"role": "user", "content": f'Transcript:\n{abridge_transcript(f_target_transcript.read(), input_char_limit)}\nInstruction\n{prompt}'})

    f_prompt.close()
    f_target_transcript.close()
    f_abridged_transcripts.close()
    return messages


In [5]:
## All Inference
metadata = open(f'{outputs_dir}/metadata.txt', 'a')
metadata.write(f"\n\n###################\n###### INFERENCE ######\n###################\n")
f_message = open(f'{outputs_dir}/messages.txt', 'w', encoding='utf-8')
temperature = 0.5
top_k = 35
top_p = 0.96
tokens_shot = 6500
tokens_target = 7500

metadata.write(f"Temperature: {temperature}\nTop_k: {top_k}\nTop_p: {top_p}\nTokens per shot: {tokens_shot}\nTokens per target: {tokens_target}\n")
metadata.close()
metadata = open(f'{outputs_dir}/metadata.txt', 'a')


## open all files in data/transcripts/processed. For each file, create a message using as many shots as given above. 
global_start = time.time()
for file in os.listdir(transcripts_dir):
    target = file.split('.')[0]
    f_message.write(f'\n###############################\n##########{target}#############\n###############################\n\n')

    if target == "catan": continue
    metadata.write(f"Processing: {target}\n")
    with open(f'{transcripts_dir}/{target}.txt', 'r', encoding='utf-8') as f_target_transcript:
        messages = build_message(shots, target, tokens_shot, tokens_target)
        for message in messages:
            f_message.write(f"####################\n{message['role']}\n####################\n{message['content']}\n")
    inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to(device)    
    metadata.write(f"    Tokens: {len(inputs[0])}\n")
    print(f"    Tokens: {len(inputs[0])}")
    ## An idea for later, do not include the oneshot prompt if we are only directly using transcript embedding
    # start_token = len(inputs[0]) 

    with torch.no_grad():
        start_time = time.time()
        text = model.generate(inputs, max_new_tokens=3500, do_sample=True, pad_token_id=tokenizer.eos_token_id, temperature=temperature, top_k=top_k, top_p=top_p)
        decoded = tokenizer.batch_decode(text)
        print(f'Decoding finished: {target} in {round(time.time() - start_time, 3)} seconds')   
        metadata.write(f"    Decoding took: {round(time.time() - start_time, 3)} seconds\n")
    ## obtain all tokens after the second "[/INST]" and remove the </s> token. Write this as our output.
    with open(f'{outputs_dir}/text/{target}.json', 'w', encoding='utf-8') as f:
        f.write(decoded[0].split('[/INST]')[-1][1:-4])

    ## Free up memory
    del text
    del decoded
    torch.cuda.empty_cache()
    print(f'cuda memory allocated: {torch.cuda.memory_allocated()/1024**3:.2f} GB', f'cuda memory cached: {torch.cuda.memory_reserved()/1024**3:.2f} GB')
    metadata.write(f'    cuda memory allocated: {torch.cuda.memory_allocated()/1024**3:.2f} GB, cuda memory cached: {torch.cuda.memory_reserved()/1024**3:.2f} GB\n')

metadata.write(f'Global runtime: {round(time.time() - global_start, 3)} seconds')
metadata.close()
f_message.close()


    Tokens: 7988
Decoding finished: typescript_fireship in 63.278 seconds
cuda memory allocated: 14.00 GB cuda memory cached: 14.36 GB
    Tokens: 7634
Decoding finished: zig in 66.503 seconds
cuda memory allocated: 14.00 GB cuda memory cached: 14.36 GB
    Tokens: 8547
Decoding finished: sql_backend in 59.555 seconds
cuda memory allocated: 14.00 GB cuda memory cached: 14.36 GB
    Tokens: 8489
Decoding finished: react_svelte in 80.618 seconds
cuda memory allocated: 14.00 GB cuda memory cached: 14.36 GB
    Tokens: 8583
Decoding finished: mixtral8x7b in 66.024 seconds
cuda memory allocated: 14.00 GB cuda memory cached: 14.36 GB
    Tokens: 7630
Decoding finished: react in 78.438 seconds
cuda memory allocated: 14.00 GB cuda memory cached: 14.36 GB
    Tokens: 8659
Decoding finished: mlops_llm_eval in 78.563 seconds
cuda memory allocated: 14.00 GB cuda memory cached: 14.36 GB
    Tokens: 7532
Decoding finished: rust in 66.729 seconds
cuda memory allocated: 14.00 GB cuda memory cached: 14

In [ ]:
## Often behavior is seen where extra characters are added in random positions. Use this function to attempt to clean them up. 
def recursive_loads(text, depth):
    if depth == 10:
        return json.loads(text)
    try:
        return json.loads(text)
    except Exception as e:
        # find "line", "column" and "char" in the error message
        error = str(e)
        line = int(error.split("line ")[1].split(" ")[0])
        column = int(error.split("column ")[1].split(" ")[0])
        char = int(error.split("char ")[1].split(" ")[0][:-1])
        print(f"line: {line}, column: {column}, char: {char}")
        
        ## delete the character at the position
        text = text[:char] + text[char+1:]
        return recursive_loads(text, depth+1)

# file_name = "sql_backend.json"
# with open(f'data/outputs/{folder_no}/text/{file_name}', 'r', encoding='utf-8') as f:
#     text = f.read()
#     print(recursive_loads(text, 0))

In [6]:
from typing import List, Literal, Annotated
from pydantic import BaseModel, field_validator, conlist, confloat

# Valid general topic names
valid_general_topics = [
    "Algorithms and Data Structures",
    "Artificial Intelligence (AI) and Machine Learning",
    "Computer Architecture",
    "Data Science and Analytics",
    "Database Systems and Management",
    "Human-Computer Interaction (HCI)",
    "Programming Languages and Software Development",
    "Software Engineering and System Design",
    "Web Development and Internet Technologies",
    "Computer Graphics and Visualization",
    "Theoretical Computer Science",
    "Quantum Computing"
]

class GeneralTopic(BaseModel):
    name: str
    ## complexity is between 0 and 1
    complexity: Annotated[float, confloat(ge=0, le=1)]

    # Validate the name to be one of the valid general topics
    @field_validator('name')
    def name_must_be_valid(cls, v):
        if v not in valid_general_topics:
            raise ValueError(f"{v} is not a valid general topic name")
        return v

class Section(BaseModel):
    title: str
    content: List[str]
    topics: List[str]

class VideoContent(BaseModel):
    introduction: str
    sections: List[Section]
    topics: List[str]
    generalTopics: List[GeneralTopic]


In [7]:
## Validate

metadata = open(f'{outputs_dir}/metadata.txt', 'a')

for file in os.listdir(f'{outputs_dir}/text'):    
    with open(f"{outputs_dir}/text/{file}") as f_transcript:
        metadata.write(f"File: {file}")
        try:
            transcript = VideoContent.parse_raw(f_transcript.read())
        except Exception as e:
            print(e)
            metadata.write(f"Errors: {e}")

metadata.close()

1 validation error for VideoContent
__root__
  Invalid \escape: line 2 column 96 (char 97) [type=value_error.jsondecode, input_value='{\n  "introduction": "Th...y": 0.58\n    }\n  ]\n}', input_type=str]
3 validation errors for VideoContent
generalTopics.0.name
  Value error, Distributed Systems is not a valid general topic name [type=value_error, input_value='Distributed Systems', input_type=str]
    For further information visit https://errors.pydantic.dev/2.6/v/value_error
generalTopics.1.name
  Value error, Consistency Hashing is not a valid general topic name [type=value_error, input_value='Consistency Hashing', input_type=str]
    For further information visit https://errors.pydantic.dev/2.6/v/value_error
generalTopics.2.name
  Value error, Replication is not a valid general topic name [type=value_error, input_value='Replication', input_type=str]
    For further information visit https://errors.pydantic.dev/2.6/v/value_error
3 validation errors for VideoContent
generalTopics.0.nam

In [8]:
# Create embeddings from outputs

def createEmbeddings(outputs_dir: str):
    """ Create embeddings from the outputs of the model
    Args:
        outputs_dir (str): The directory containing the outputs
    """
    
    metadata = open(f'{outputs_dir}/metadata.txt', 'a')

    for file in os.listdir(f'{outputs_dir}/text'):
        print(f'cuda memory allocated: {torch.cuda.memory_allocated()/1024**3:.2f} GB', f'cuda memory cached: {torch.cuda.memory_cached()/1024**3:.2f} GB')

        target = file.split('.')[0]
        start_time = time.time()
        with open(f'{outputs_dir}/text/{target}.json', 'r', encoding='utf-8') as f:
            inputs = tokenizer(f.read(), return_tensors="pt").to(device)     
        with torch.no_grad():
            hidden = model(**inputs, output_hidden_states=True)
        print(f'Hidden states finished: {target} in {round(time.time() - start_time, 3)} seconds')
        metadata.write(f"Target: {target}, Hidden states took: {round(time.time() - start_time, 3)} seconds\n")

        ## Write hidden states
        ## Cut off all tokens before the first start token to remove oneshot and prompt.
        tensor_t = hidden.hidden_states[-1].transpose(1,2)
        # max_pool_t = torch.nn.functional.max_pool1d(tensor_t, tensor_t.shape[2])
        # avg_pool_t = torch.nn.functional.avg_pool1d(tensor_t, tensor_t.shape[2])
        max_pool = torch.nn.functional.max_pool1d(tensor_t, tensor_t.shape[2]).transpose(1, 2).squeeze()
        avg_pool = torch.nn.functional.avg_pool1d(tensor_t, tensor_t.shape[2]).transpose(1, 2).squeeze()
        print(max_pool.shape)
        torch.save(max_pool, f'{outputs_dir}/embeddings/max_{target}.pt')
        torch.save(avg_pool, f'{outputs_dir}/embeddings/avg_{target}.pt')

        print(f'Target: {target} finished. Wrote to file.')
        metadata.write(f'    Target: {target} tensor finished. Wrote to file.\n')

        del inputs
        del hidden
        del tensor_t
        del max_pool
        del avg_pool
        print(f'cuda memory allocated: {torch.cuda.memory_allocated()/1024**3:.2f} GB', f'cuda memory cached: {torch.cuda.memory_cached()/1024**3:.2f} GB')
        torch.cuda.empty_cache()

    metadata.close()

In [9]:
## Run to obtain embeddings
createEmbeddings(f'data/outputs/{folder_no}')


## LETS CREATE A TON
# for str_num in os.listdir('data/outputs'):
#     num = int(str_num)
#     if num < 11: continue # only use recent 
    
#     os.makedirs(f'data/outputs/{num}/embeddings', exist_ok=True)

#     # if embeddings dir is empty
#     if len(os.listdir(f'data/outputs/{num}/embeddings')) == 0:
#         createEmbeddings(f'data/outputs/{num}')

cuda memory allocated: 14.00 GB cuda memory cached: 14.36 GB


/home/ubuntu/.local/lib/python3.10/site-packages/torch/cuda/memory.py:440: FutureWarning: torch.cuda.memory_cached has been renamed to torch.cuda.memory_reserved
  warnings.warn(


Hidden states finished: rust in 0.275 seconds
torch.Size([4096])
Target: rust finished. Wrote to file.
cuda memory allocated: 14.00 GB cuda memory cached: 14.73 GB
cuda memory allocated: 14.00 GB cuda memory cached: 14.36 GB
Hidden states finished: sql_backend in 0.235 seconds
torch.Size([4096])
Target: sql_backend finished. Wrote to file.
cuda memory allocated: 14.00 GB cuda memory cached: 14.67 GB
cuda memory allocated: 14.00 GB cuda memory cached: 14.36 GB
Hidden states finished: react_svelte in 0.309 seconds
torch.Size([4096])
Target: react_svelte finished. Wrote to file.
cuda memory allocated: 14.00 GB cuda memory cached: 14.84 GB
cuda memory allocated: 14.00 GB cuda memory cached: 14.36 GB
Hidden states finished: full-stack in 0.241 seconds
torch.Size([4096])
Target: full-stack finished. Wrote to file.
cuda memory allocated: 14.00 GB cuda memory cached: 14.72 GB
cuda memory allocated: 14.00 GB cuda memory cached: 14.36 GB
Hidden states finished: hashing in 0.238 seconds
torch.Siz

In [ ]:
### catan.txt
## Clean transcript, with new lines
## Char: 9388, Word: 1840, Tokens: 2483, Runtime: 3m 4.2s on Pytorch MPS, T/S = 13.5

## Clean transcript, no new lines
## Char: 9388, Word: 1840, Tokens: 2174, Runtime: 2m 43.9s on Pytorch MPS, T/S = 13.25

## Clean transcript, charging
## Char: 9388, Word: 1840, Tokens: 2174, Runtime: 2m 32.8s on Pytorch MPS

#### On A10s
# 8,000-token limit for Mistral-7B

### catan.txt
## Tokens: 2174, Runtime: 48.8s, T/S = 44.55

### mixtral8x7b.txt
## Tokens: 16981, Runtime: 13m 8.4s, T/S = 21.59

### mlops_llm_eval.txt
## Tokens: 10993, Runtime: 7m 5.1s, T/S = 25.87

### typescript_fireship.txt
## Tokens: 1042, Runtime: 27.6s, T/S = 37.75

### localized_deployment.txt | Tokens: 892,
## A10 // Runtime: 21.6s, T/S = 41.3
## M1  // Runtime:
